<a href="https://colab.research.google.com/github/Chahethsen12/The_Cloud_Brain_Agent/blob/main/The_Cloud_Brain_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Libraries

In [1]:
!pip install -q langchain langchain-groq langgraph tavily-python
print("Libraries installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 6.7 MB/s eta 0:00:00
Libraries installed successfully!


Set API Keys

In [ ]:
import os
from google.colab import userdata
from getpass import getpass

# Enter your keys when prompted.
# It keeps them hidden so they don't appear in your code history.
os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API Key: ")
os.environ["TAVILY_API_KEY"] = getpass("Enter your Tavily API Key: ")

The Agent Code

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults

# 1. SETUP TOOLS & MODEL
tool = TavilySearchResults(max_results=2)
llm = ChatGroq(model="llama3-8b-8192")
llm_with_tools = llm.bind_tools([tool])

# 2. DEFINE STATE
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 3. DEFINE NODES
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

class BasicToolNode:
    def __init__(self, tools: list) -> None:
        self.tools_by_name = {t.name: t for t in tools}
    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(tool_call["args"])
            outputs.append({
                "tool_call_id": tool_call["id"],
                "role": "tool",
                "name": tool_call["name"],
                "content": str(tool_result)
            })
        return {"messages": outputs}

tool_node = BasicToolNode(tools=[tool])

# 4. BUILD GRAPH
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)
graph_builder.add_edge(START, "chatbot")

def route_tools(state: State):
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        return END
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return END

graph_builder.add_conditional_edges("chatbot", route_tools, {"tools": "tools", END: END})
graph_builder.add_edge("tools", "chatbot")
graph = graph_builder.compile()

print("Agent built successfully! Run the next cell to chat.")

Run the Chat

In [ ]:
user_input = "What are the latest updates on OpenAI's o1 model from 2024-2025?"
events = graph.stream({"messages": [("user", user_input)]}, stream_mode="values")

for event in events:
    if "messages" in event:
        print(f"🤖 Agent: {event['messages'][-1].content}")
        print("-" * 50)